In [243]:
#import datasets
from aif360.datasets import StandardDataset
from aif360.datasets import CompasDataset
#import fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt


#### 0. COMPAS Dataset used as the starting point to get predictions from.

In [244]:
#load COMPAS dataset

try:
    compas = CompasDataset(
        protected_attribute_names=['sex', 'race'],
        privileged_classes=[['Female'], ['Caucasian']], 
        features_to_keep=['age', 'c_charge_degree', 'race', 'age_cat', 
                          'sex', 'priors_count', 'days_b_screening_arrest', 'c_charge_desc'],
        features_to_drop=[],
        categorical_features=['age_cat', 'c_charge_degree', 'c_charge_desc'],
        label_name='two_year_recid'
    )
    print("Dataset loaded successfully!")

    #returns the dataframe and the metadata in a tuple using a function from the compas dataset
    df, meta = compas.convert_to_dataframe()

except Exception as e:
    print(f"Error loading COMPAS dataset: {e}")

Dataset loaded successfully!


#### 0.1 Train a model with the COMPAS dataset 

- 80/20 train/test split
- key point is the extraction of the group membership from this dataset to create an array storing group membership for each instance.

In [245]:
# copy dataset to ensure original remains unchanged
df = df.copy()
#print(df)

#separate features and labels
features = ['race', 'sex', 'priors_count', 'c_charge_degree=F', 'c_charge_degree=M']
target = 'two_year_recid' #binary target where 0 means does not offend, 1 means offends

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

X_test_indices = X_test.index

#retrive each instance's group membership before scaling X_test to make predictions 
#scaling will make it lose the index information to retrieve this information
grp_membership = df.loc[X_test_indices, 'race'].values
print("\n group membership: ", grp_membership, "\n")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(solver='lbfgs')
model.fit(X_train_scaled, y_train)

#predicted class labels 0 or 1
y_pred = model.predict(X_test_scaled)


 group membership:  [1. 1. 0. ... 0. 0. 0.] 



#### 1. From the above classifier, extract three arrays that are inputs for new_metric function

To do: Have three arrays each for predictions, groud truth labels, and group membership from the dataset and model.

Important aspect of this part: indices of each array needs to align such that pred[0] refers to ground_truth[0] and grp_membership[0]. 

The arrays:
1. predictions: positive and negative predictions from the classifier.
2. ground_truth: this is the two_yr_recid column that represents the target variables. So, use y_test which contains ground truth values from the train_test_split for the dataset.
3. grp_membership: array of group membership for each instance containing privileged (label 1 - Caucasian) and non-privileged (label 0 - not Caucasian) as defined by protected attribute of 'race'.


In [246]:
#predictions array
predictions = y_pred 
print("predictions: ", predictions)

#ground truth labels array
ground_truth = y_test.values 
print("ground truth labels: ", ground_truth)

#group membership array defined above where model is trained.
print("group membership: ", grp_membership)

predictions:  [0. 0. 0. ... 0. 0. 0.]
ground truth labels:  [0. 0. 1. ... 0. 1. 1.]
group membership:  [1. 1. 0. ... 0. 0. 0.]


#### 2. Function for the new metric using the three arrays as input.

- The main conceptual idea behind this metric: as a basic starting point, create a classifier level group metric that takes into consideration individual components.
- Assumption: This will allow for a quantification of fairness in a more comprehensive manner. 
- Evaluate behaviour by: giving the function varied distributions of arrays as input.

In [252]:
def new_metric(arr_pred, arr_true, arr_grp): 
    
    #Two arrays for privileged and not privileged 
    #g1 and g2- contain predictions and trues are lists of lists [[], []]
    grp_priv = [[], []]
    grp_unpriv = [[], []]

    #j is the number of unique groups in arr_grp 
    j = len(set(arr_grp)) #!an implicit parameter.
    
    #print("total number of unique groups: ", j)

    for i, label in enumerate(arr_grp):
        #for privileged class
        if label == 1.0:
            #add the corresponding prediction + gt label for that class using the index associated with that label
            grp_priv[0].append(arr_pred[i])
            grp_priv[1].append(arr_true[i])
        
        #for unprivileged class
        else:
            grp_unpriv[0].append(arr_pred[i])
            grp_unpriv[1].append(arr_true[i])
    
    #print("Privileged group: ", grp_priv)
    #print("Unprivileged group: ", grp_unpriv)
    
    priv_indiv_bi = [] #stores individual benefit value of each instance
    priv_grp_bi = 0 #tracks total benefit for group
    
    #1. for each index in a group calculate the benefit, bi
    for pred, gt in zip(grp_priv[0], grp_priv[1]):
        #the individual component from GEI to calculate benefit for each instance in a group
        # original bi calculation with original range of 0,1,2
        indiv_benefit = (int(pred) - int(gt)) + 1
        
        #2. Sum the total benefit of each group
        priv_grp_bi += indiv_benefit
        
        #3. divide by size of group 1 - result of this is for each class
        priv_av_bi = priv_grp_bi / len(grp_priv[0]) #this is the total number of instances in each group. [0] has predictions which will give that number

        #store individual benefit of each instance in a list
        priv_indiv_bi.append(indiv_benefit)

    #print(priv_grp_bi)
    # print(priv_av_bi)
    #print("all bi scores for privileged instances:\n", priv_indiv_bi)

    unpriv_indiv_bi = []
    unpriv_grp_bi = 0
    for pred, gt in zip(grp_unpriv[0], grp_unpriv[1]):
        indiv_benefit = (int(pred) - int(gt)) + 1
        unpriv_grp_bi += indiv_benefit
        unpriv_av_bi = unpriv_grp_bi / len(grp_priv[0])
        unpriv_indiv_bi.append(indiv_benefit)
        
    #print(unpriv_grp_bi)
    #print(unpriv_av_bi)
    #print("all bi scores for unprivileged instances:\n", unpriv_indiv_bi)

    #4. division result is divided by the sum of g1 and g2 - J
    result = (priv_av_bi + unpriv_av_bi) / j

    #print("new metric value: ", result)
    #return priv_av_bi, unpriv_av_bi to see individual benefit for each group in arr_grp
    return result

#print("results from COMPAS data classification:\n", new_metric(predictions, ground_truth, grp_membership))


#### Below are test cases being given as input to the metric function.

- it returns a score of the final calculation result = (priv_av_bi + unpriv_av_bi) / j
- this is an average of the average bi values for priviliged and unprivileged groups of the protected label.
- Also printing the list of individual bi scores for each instance in the group array.

In [253]:
# test case 1: arr_grp1_asc, arr_true1, arr_pred1 
print("results from test case 1 (random classifier) - all arrays with 50/50:\n", new_metric(arr_grp1_asc, arr_true1, arr_pred1) )

# # test case 2: arr_grp1_desc, arr_true1_asc, arr_pred1_asc 
# print("results from test case 2 - all arrays with 50/50 but arr_grp has 1s before 0s:\n", new_metric(arr_grp1_desc, arr_true1_asc, arr_pred1_asc))

# # test case 3: arr_grp2, arr_true2, arr_pred2 
# print("results from test case 3 - arr_true is 80/20, rest are 50/50:\n", new_metric(arr_grp2, arr_true2, arr_pred2) )

# # test case 4: arr_grp3, arr_true3, arr_pred3 
# print("results from test case 4 - arr_pred is 80/20, rest are 50/50:\n", new_metric(arr_grp3, arr_true3, arr_pred3))

# #test case 5:
# print("results from test case 5:\n", new_metric(arr_grp4, arr_true4, arr_pred4) )

# #test case 6:
# print("results from test case 6:\n", new_metric(arr_grp5, arr_true5, arr_pred5) )

# #test case 7:
# print("results from test case 7:\n", new_metric(arr_grp6, arr_true6, arr_pred6) )

# #test case 8: 
# print("results from test case 8 :\n", new_metric(arr_grp7, arr_true7, arr_pred7) )

# #test case 9:
# print("results from test case 9 :\n", new_metric(arr_grp8, arr_true8, arr_pred8))

results from test case 1 (random classifier) - all arrays with 50/50:
 1.0


#### 3. Simulate classification results for edge cases and expand table with these numbers

- to see if simulated classifier does good on one group and not on other (priv vs unpriv groups).
- creating arrays with different distrubutions.
- wanting to see if my metric gives more intuitive and interpretable results.
- test cases where metrics act differently using the fake distributions acting as output from classifiers.

#### 3.1 Make a function to generate different array distributions. 

#### Settings:
##### Explanation of swap function:
- the tuple of probabilities (0, 1) means [0] is the probability for 0s and [1] for 1s. 
- If a swap flag is set to True (swap_group=True), the tuple is reversed so that the first element represents the probability for 1.
- the effect this has: It changes the interpretation of the distribution tuple as just giving it the distribution e.g. "80/20" does not specify 80% 0s or 1s, or 20% 0s or 1s.
- For example, the default is set to (0.8, 0.2) meaning 80% 0s and 20% 1s. If swap=True, it becomes 80% 1s and 20% 0s.
- This affects how many 0s versus 1s are generated.
- Why do this: To see if it impacts metric behaviour.
- e.g. if we have an array arr_grp=[1111111100] and arr_pred=[1111100000], arr_true=[1111100000], then the unprivileged group would get true negative outcome.
- if arr_grp=[0011111111], then the outcome would switch to true positive. This may have an impact on the metric score. 

##### How is swap different from order:
- It decides how the instances are arranged in the final array.
- It determines whether the array is ordered with all 0s first (ascending) or all 1s first (descending) after the counts have been set.

In [299]:
#function to generate arrays synthetically by using fixed distributions:

#function takes in the size of array and the type of distribution to generate
#also takes as input a flag to swap the default mapping of probabilities associated with the distribution
#also takes as input the order for correct 0s and 1s placement.
def gen_fixed_dist_combinations(num_of_instances, grp_dist, true_dist, pred_dist, 
                          pred_order='asc', true_order='asc', grp_order='asc', 
                          swap_group=False, swap_gt=False, swap_pred=False):
    
    #a dictionary that stores the types of expected distributions and maps to their corresp probabilties
    distribution_mapping = {"50/50": (0.5, 0.5), 
                            "80/20": (0.8, 0.2), 
                            "90/10": (0.9, 0.1), 
                            "20/80": (0.2, 0.8), 
                            "10/90":(0.1, 0.9)}

    #get the given parameters of probabilties for each distribution from the dict
    group_probability = distribution_mapping.get(grp_dist)
    #print("original group probs: ", group_probability)
    gt_probability = distribution_mapping.get(true_dist)
    pred_probability = distribution_mapping.get(pred_dist)
   

    #if swap parameter is True, then it means that the first element in the dict is interpreted as being the probability for 1s
    # if swap_group:
    #     group_probability = (group_probability[1], group_probability[0])
    #     #print("swapped group prob: ", group_probability)
    # if swap_gt:
    #     gt_probability = (gt_probability[1], gt_probability[0])
    # if swap_pred:
    #     pred_probability = (pred_probability[1], pred_probability[0])

    #calculations for fixing the 0s and 1s for each array using probabilities and array size
    group_zeroes =  int(num_of_instances * group_probability[0])
    group_ones =  num_of_instances - group_zeroes

    true_zeroes = int(num_of_instances * gt_probability[0])
    true_ones = num_of_instances - true_zeroes

    pred_zeroes = int(num_of_instances * pred_probability[0])
    pred_ones = num_of_instances - pred_zeroes

    #create predictions array based on desired order of 0s and 1s.
    if pred_order == 'asc':
        arr_pred = np.array([0] * pred_zeroes + [1] * pred_ones)
    elif pred_order == 'desc':
        arr_pred = np.array([1] * pred_zeroes + [0] * pred_ones)
    else:
        raise ValueError("prediction array order must be 'asc' or 'desc'")

    #create ground truth labels array based on desired order of 0s and 1s.
    if true_order == 'asc':
        arr_true = np.array([0] * true_zeroes + [1] * true_ones)
    elif pred_order == 'desc':
        arr_true = np.array([1] * true_zeroes + [0] * true_ones)
    else:
        raise ValueError("prediction array order must be 'asc' or 'desc'")

    #create group memberships array based on desired order of 0s and 1s.
    if grp_order == 'asc':
        arr_grp = np.array([0] * group_zeroes + [1] * group_ones)
    elif grp_order == 'desc':
        arr_grp = np.array([1] * group_zeroes + [0] * group_ones)
    else:
        raise ValueError("group membership array order must be 'asc' or 'desc'")
    
    return arr_grp, arr_true, arr_pred  

# print("predictions: ", arr_pred)
# print("grount truth labels: ", arr_true)
# print("group membership is 50/50: ", arr_grp_asc)


### Function to calculate the Balanced Accuracy of each array
- need the true and predicted pair of arrays
- Aim: To have this score as an anchor point to get an idea of positive and negative outcomes from a classifier.

In [301]:
from sklearn.metrics import balanced_accuracy_score

def balanced_accuracy(arr_true, arr_pred):
    y_true = arr_true
    y_pred = arr_pred
    return balanced_accuracy_score(y_true, y_pred)

### Function to return a metric object for AIF360 framework
- function returns a metric object that is compatible for the AIF360 framework
- the object can be created in the 'automate_analysis' function to then apply AIF360 metrics to.

#### Experiment using these metrics: Conducting a comparison of synthetic classification results with evaluation metrics in AIF360 

- The goal here is to be able to use the existing metrics from aif360 and get a value for each array distribution.
- Outcome: to have a comparison of my metric with existing metrics and be able to argue that the custom metric is somehow more interpretable and than existing ones. Therefore, making it a more comprehensible metric. 
- Issue: not bring able to give correct inputs to the functions that are needed to apply metrics within the framework.
- Possible solution to do later: use the tutorials as example to see how they give input to these functions and perhaps fake the datasets with the fake arrays to give them as input.

In [302]:
from aif360.datasets import BinaryLabelDataset 
from aif360.metrics import ClassificationMetric

def aif360_metric_object(num_of_instances, arr_grp, arr_true, arr_pred, seed=42):
    
    # synthetic feature data just to comply with AIF360 formatting to apply metric.
    np.random.seed(seed)
    features = pd.DataFrame({
        'feature1': np.random.rand(num_of_instances),
        'feature2': np.random.rand(num_of_instances),
        'race': np.random.randint(0, 2, num_of_instances)  # placeholder protected attribute
    })

    features['race'] = arr_grp #protected attribute to represent the group membership array
    
    #these will be the variables to store the generated arrays with varying distributions 
    #these changing arrays will show the changing score of each metric being applied 

    data_true = features.copy() #dataframe with true labels
    data_true['label'] = arr_true
    
    data_pred = features.copy() #dataframe with predicted labels
    data_pred['label'] = arr_pred
    
    # Create BinaryLabelDataset objects for true and predicted datasets
    dataset_true = BinaryLabelDataset(df=data_true, label_names=['label'], protected_attribute_names=['race'])
    dataset_pred = BinaryLabelDataset(df=data_pred, label_names=['label'], protected_attribute_names=['race'])
    
    privileged_groups = [{'race': 1}]  # represents the majority group
    unprivileged_groups = [{'race': 0}]  # represents the minority group
    
    metric = ClassificationMetric(dataset_true, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    return metric
    
#aif360_metric = aif360_metric_object(num_of_instances, arr_grp, arr_true, arr_pred)

#### Function to automate analysis process

- function needs to be able to create different combinations of distributions to give as input to the function that generates arrays.
- store the generated arrays in variables
- give those variables as input to the custom metrics and store the score in a variable
- give pred and true arrays as input to other exisitng metrics and get a score in variables that correspond to each metric.
- get BAC similarly
- have a list of scores associated with that distribution
- add that as a row to the pandas dataframe and export it to latex
- use the distribution combination as the naming scheme for the 'approach' that has all the scores associated with it.
- returns the dataframe - a table showing the scores for each metric given  arrays with different distributions.

In [303]:
from aif360.datasets import BinaryLabelDataset 
from aif360.metrics import ClassificationMetric

def automate_analysis(num_of_instances):
  
    dist_types = ["50/50", "80/20", "90/10", "20/80", "10/90"]

    results = []

    for grp_dist in dist_types:
        for true_dist in dist_types:
            for pred_dist in dist_types:
                #generate array for current combination
                arr_grp, arr_true, arr_pred = gen_fixed_dist_combinations(num_of_instances, 
                                                                          grp_dist, 
                                                                          true_dist,
                                                                          pred_dist)

                balanced_accuracy_score = balanced_accuracy(arr_true, arr_pred)
                standard_custom_metric = new_metric(arr_grp, arr_true, arr_pred)
                
                
                aif360_metric = aif360_metric_object(num_of_instances, arr_grp, arr_true, arr_pred, seed=42)
                gei_score = aif360_metric.generalized_entropy_index()
                statistical_parity_diff = aif360_metric.mean_difference()
                disparate_impact = aif360_metric.disparate_impact()
                eq_opp_diff =  aif360_metric.equal_opportunity_difference()
                av_odds_diff = aif360_metric.average_odds_difference()
                
                results.append({
                    "grp_dist":grp_dist,
                    "true_dist": true_dist,
                    "pred_dist": pred_dist,
                    "balanced accuracy score": balanced_accuracy_score,
                    "standard_custom_metric score": standard_custom_metric,                    
                    "gei_score": gei_score,
                    "statistical_parity_diff":statistical_parity_diff,
                    "disparate_impact": disparate_impact,
                    "eq_opp_diff":eq_opp_diff, 
                    "av_odds_diff":av_odds_diff 
                })

    #create pandas dataframe from the results list
    metrics_df = pd.DataFrame(results)
    return metrics_df

num_of_instances = 100
metrics_scores_table = automate_analysis(num_of_instances)
print(metrics_scores_table)

latex_table = metrics_scores_table.to_latex(index=False)
#print(latex_table)
                

    grp_dist true_dist pred_dist  balanced accuracy score  \
0      50/50     50/50     50/50                 1.000000   
1      50/50     50/50     80/20                 0.700000   
2      50/50     50/50     90/10                 0.600000   
3      50/50     50/50     20/80                 0.700000   
4      50/50     50/50     10/90                 0.600000   
..       ...       ...       ...                      ...   
120    10/90     10/90     50/50                 0.777778   
121    10/90     10/90     80/20                 0.611111   
122    10/90     10/90     90/10                 0.555556   
123    10/90     10/90     20/80                 0.944444   
124    10/90     10/90     10/90                 1.000000   

     standard_custom_metric score  gei_score  statistical_parity_diff  \
0                        1.000000   0.000000                -1.000000   
1                        2.500000   0.214286                -0.400000   
2                        5.000000   0.333333    

In [272]:
#function to save each generated array to a numpy zip file:

def store_arrays(file, arr_grp, arr_true, arr_pred):
    #save three arrays to a .npz file with keys 'group membership', 'ground truth', 'predictions'
    np.savez(file, group=arr_grp, true=arr_true, pred=arr_pred)
    print(f"Arrays saved to {file}")

store_arrays("distributions.npz", arr_grp, arr_true, arr_pred)

loaded_arrays = np.load('distributions.npz')
loaded_arrays.files
#loaded_arrays['group']


Arrays saved to distributions.npz


['group', 'true', 'pred']

### Plot the scores being calculated